In [3]:
import pandas as pd
import gensim
from gensim.models import Word2Vec
from nltk.stem import WordNetLemmatizer
import numpy as np

In [4]:
import nltk
import re

In [5]:
#dlr = nltk.downloader.Downloader()
#dlr._update_index()
#dlr.download('wordnet')
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('omw-1.4')
from nltk.corpus import stopwords

In [6]:
data_loc = 'data/'

In [7]:
data1 = pd.read_csv('winemag-data_first150k.csv', index_col=0)
data2 = pd.read_csv('winemag-data-130k-v2.csv', index_col=0)

In [8]:
#Panen andmed kokku
dataset = pd.concat([data1,data2])

In [9]:
dataset.reset_index(drop=True)

,country,description,designation,points,price,province,region_1,region_2,variety,winery,taster_name,taster_twitter_handle,title
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz,NaN,NaN,NaN
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez,NaN,NaN,NaN
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley,NaN,NaN,NaN
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,NaN,NaN,NaN
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
280896,Germany,Notes of honeysuckle and cantaloupe sweeten th...,Brauneberger Juffer-Sonnenuhr Spätlese,90,28.0,Mosel,NaN,NaN,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef),Anna Lee C. Iijima,NaN,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...
280897,US,Citation is given as much as a decade of bottl...,NaN,90,75.0,Oregon,Oregon,Oregon Other,Pinot Noir,Citation,Paul Gregutt,@paulgwine,Citation 2004 Pinot Noir (Oregon)
280898,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,NaN,Gewürztraminer,Domaine Gresser,Roger Voss,@vossroger,Domaine Gresser 2013 Kritt Gewurztraminer (Als...
280899,France,"A dry style of Pinot Gris, this is crisp with ...",NaN,90,32.0,Alsace,Alsace,NaN,Pinot Gris,Domaine Marcel Deiss,Roger Voss,@vossroger,Domaine Marcel Deiss 2012 Pinot Gris (Alsace)


In [10]:
#Vaatan, mis toimub
dataset.head()

,country,description,designation,points,price,province,region_1,region_2,variety,winery,taster_name,taster_twitter_handle,title
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz,NaN,NaN,NaN
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez,NaN,NaN,NaN
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley,NaN,NaN,NaN
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,NaN,NaN,NaN
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude,NaN,NaN,NaN


In [23]:
new_stopwords = dataset.country.astype(str).unique().tolist()
new_stopwords = [x.lower() for x in new_stopwords]
#new_stopwords[0:5]

['us', 'spain', 'france', 'italy', 'new zealand']

In [12]:
stop_phrases = dataset.variety.astype(str).unique().tolist()
stop_phrases = [x.lower() for x in stop_phrases]

In [42]:
country_phrases = [x for x in new_stopwords if " " in x]

In [44]:
stop_phrases.extend(country_phrases)

In [47]:
new_stopwords = list(filter(lambda x: x not in country_phrases, new_stopwords))

In [81]:
variety_words = [x for x in stop_phrases if " " not in x]

In [84]:
new_stopwords.extend(variety_words)

In [85]:
stop_phrases = list(filter(lambda x: x not in variety_words, stop_phrases))

In [55]:
lemmatizer = WordNetLemmatizer()

In [56]:
desc = dataset.description

In [57]:
reviews_lower = [x.lower() for x in desc]

In [58]:
reviews_lower_cl = [re.sub('[(.,!?)]', '', x) for x in reviews_lower]

In [59]:
reviews_lower_cl[0]

'this tremendous 100% varietal wine hails from oakville and was aged over three years in oak juicy red-cherry fruit and a compelling hint of caramel greet the palate framed by elegant fine tannins and a subtle minty tone in the background balanced and rewarding from start to finish it has years ahead of it to develop further nuance enjoy 2022–2030'

In [60]:
reviews_sent = [nltk.sent_tokenize(x) for x in reviews_lower_cl]

In [61]:
reviews_sent = [y for x in reviews_sent for y in x]

In [62]:
reviews_sent[0]

'this tremendous 100% varietal wine hails from oakville and was aged over three years in oak juicy red-cherry fruit and a compelling hint of caramel greet the palate framed by elegant fine tannins and a subtle minty tone in the background balanced and rewarding from start to finish it has years ahead of it to develop further nuance enjoy 2022–2030'

In [63]:
reviews_words = [nltk.word_tokenize(x) for x in reviews_sent]

In [64]:
reviews_words[0]

['this',
 'tremendous',
 '100',
 '%',
 'varietal',
 'wine',
 'hails',
 'from',
 'oakville',
 'and',
 'was',
 'aged',
 'over',
 'three',
 'years',
 'in',
 'oak',
 'juicy',
 'red-cherry',
 'fruit',
 'and',
 'a',
 'compelling',
 'hint',
 'of',
 'caramel',
 'greet',
 'the',
 'palate',
 'framed',
 'by',
 'elegant',
 'fine',
 'tannins',
 'and',
 'a',
 'subtle',
 'minty',
 'tone',
 'in',
 'the',
 'background',
 'balanced',
 'and',
 'rewarding',
 'from',
 'start',
 'to',
 'finish',
 'it',
 'has',
 'years',
 'ahead',
 'of',
 'it',
 'to',
 'develop',
 'further',
 'nuance',
 'enjoy',
 '2022–2030']

In [65]:
for i in range(len(reviews_words)):
    reviews_words[i] = [lemmatizer.lemmatize(w) for w in reviews_words[i]]

In [116]:
reviews_lemmatized = [' '.join(x) for x in reviews_words]

In [117]:
reviews_lemmatized[0]

'this tremendous 100 % varietal wine hail from oakville and wa aged over three year in oak juicy red-cherry fruit and a compelling hint of caramel greet the palate framed by elegant fine tannin and a subtle minty tone in the background balanced and rewarding from start to finish it ha year ahead of it to develop further nuance enjoy 2022–2030'

In [70]:
stop_phrases[0]

'cabernet sauvignon'

In [72]:
#To check if stop phrase removal actually works, we find one description with stop phrase "cabernet sauvignon" and its index
for i in range(len(reviews_lemmatized)):
    if stop_phrases[0] in reviews_lemmatized[i]:
        print(reviews_lemmatized[i])
        print("Index:", i)
        break

this savory blend of 40 % merlot 30 % cabernet sauvignon 25 % cabernet franc and 5 % syrah open with aroma of roasted coffee bean leather tobacco and dark woodland berry the chewy palate offer juicy plum cassis and clove alongside polished tannin drink through 2019
Index: 59


In [118]:
#Removing stop phrases
for x in stop_phrases:
    for i in range(len(reviews_lemmatized)):
        reviews_lemmatized[i] = re.sub(x, "", reviews_lemmatized[i])

In [119]:
reviews_lemmatized[59]

'this savory blend of 40 % merlot 30 %  25 %  and 5 % syrah open with aroma of roasted coffee bean leather tobacco and dark woodland berry the chewy palate offer juicy plum cassis and clove alongside polished tannin drink through 2019'

In [120]:
#Further cleanup
cleaned_reviews = [re.sub('[^a-zA-Z-]', ' ', x) for x in reviews_lemmatized]
cleaned_reviews = [re.sub(r'\s+', ' ', x) for x in cleaned_reviews]

In [121]:
cleaned_reviews[59]

'this savory blend of merlot and syrah open with aroma of roasted coffee bean leather tobacco and dark woodland berry the chewy palate offer juicy plum cassis and clove alongside polished tannin drink through '

In [122]:
#Tokenizing the descriptions again
reviews = [nltk.word_tokenize(x) for x in cleaned_reviews]

In [123]:
reviews[59]

['this',
 'savory',
 'blend',
 'of',
 'merlot',
 'and',
 'syrah',
 'open',
 'with',
 'aroma',
 'of',
 'roasted',
 'coffee',
 'bean',
 'leather',
 'tobacco',
 'and',
 'dark',
 'woodland',
 'berry',
 'the',
 'chewy',
 'palate',
 'offer',
 'juicy',
 'plum',
 'cassis',
 'and',
 'clove',
 'alongside',
 'polished',
 'tannin',
 'drink',
 'through']

In [124]:
stop_words = stopwords.words('english')
stop_words.extend(new_stopwords)

In [125]:
for i in range(len(reviews)):
    reviews[i] = [w for w in reviews[i] if w not in stop_words and len(w) > 2 if w != 'no']

In [126]:
reviews[59]

['savory',
 'blend',
 'open',
 'aroma',
 'roasted',
 'coffee',
 'bean',
 'leather',
 'tobacco',
 'dark',
 'woodland',
 'berry',
 'chewy',
 'palate',
 'offer',
 'juicy',
 'plum',
 'cassis',
 'clove',
 'alongside',
 'polished',
 'tannin',
 'drink']

In [ ]:
model = Word2Vec(reviews, vector_size = 200 , window = 4 , min_count = 1,
                               seed = 1000, hs = 0, negative = 10,workers=16, epochs = 25)

In [ ]:
model.save("word2vec.model")

In [117]:
model.wv.most_similar('sweet')

[('candied', 0.5667949914932251),
 ('luscious', 0.5449540615081787),
 ('sweetness', 0.5442702174186707),
 ('upfront', 0.5179322361946106),
 ('jam', 0.5063546299934387),
 ('sweetly', 0.5018275380134583),
 ('sweetened', 0.48643139004707336),
 ('syrup', 0.47206464409828186),
 ('sugary', 0.4624304175376892),
 ('baked', 0.4607876241207123)]

In [118]:
model.wv.most_similar('dry')

[('dryness', 0.45747560262680054),
 ('bite', 0.45368340611457825),
 ('marked', 0.40880826115608215),
 ('softly', 0.3971874713897705),
 ('gentle', 0.3944217264652252),
 ('yet', 0.38916701078414917),
 ('drily', 0.3874289393424988),
 ('flavored', 0.3835741877555847),
 ('steely', 0.38233959674835205),
 ('brisk', 0.3813340961933136)]